In [1]:
from pathlib import Path 
from bs4 import BeautifulSoup
from urllib.request import urlopen
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import json

In [2]:
# get service key from json file
with open('service_key.json', 'r') as f:
    service_key = json.load(f)
    service_key = service_key['key']

In [3]:
update_date = datetime.now().strftime("%y%m%d%H%M%S")

output_dir = Path("../datasets")
output_file = Path("haccp_dataset_{}.csv".format(update_date))
if not Path.is_dir(output_dir):
    Path.mkdir(output_dir)
output_path = output_dir / output_file

In [4]:
url = "http://apis.data.go.kr/B553748/CertImgListService/getCertImgListService?\
ServiceKey={service_key}&\
returnType=xml&\
pageNo={page_no}&\
numOfRows={num_row}"

data = urlopen(url.format(service_key = service_key, page_no = 1, num_row = 1)).read()
soup = BeautifulSoup(data, "html.parser")
num_page = int(soup.find('totalcount').text) // 100 + 1

tags = [tag.name for tag in soup.find('item').find_all()]
tags_dict = {'{}'.format(tag) : [] for tag in tags}

In [5]:
for i in tqdm(range(1, num_page + 1)):
    data = urlopen(url.format(service_key = service_key, page_no = i, num_row = 100)).read()
    soup = BeautifulSoup(data, "html.parser")
    items = soup.find("items")
    
    for item in items.findAll("item"):
        for tag in tags:
            res = item.find(tag)
            val = res.text.strip() if res != None else "None"
            tags_dict[tag].append(val)

df = pd.DataFrame.from_dict(tags_dict)
df.to_csv(output_path, encoding='utf-8-sig')